In [43]:
import pandas as pd
import json
from docx import Document
from sentence_transformers import SentenceTransformer

# 1. 설정 및 경로
PATHS = [
    ("SKT", r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\요금제RAG\SKT요금제정보_251209.docx"),
    ("KT", r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\요금제RAG\KT요금제정보_251209.docx"),
    ("LGU+", r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\요금제RAG\LGU+요금제정보_260102.docx"),
]


In [39]:
# 2. 텍스트 분리(Parsing) 로직
carrier_plans = {"SKT": [], "LGU+": [], "KT": []}

print("📄 통신사별 텍스트 분리 시작...")

for carrier_name, path in PATHS:
    doc = Document(path)
    paras = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    
    current_block = []
    
    for para in paras:
        is_new_section = False
        
        # 조건 1: "요금제란?" 키워드가 포함된 문단 (설명 섹션의 시작)
        if "요금제란?" in para:
            is_new_section = True
            
        # 조건 2: "[통신사명]" 태그로 시작하는 문단 (상세 정보 섹션의 시작)
        elif para.startswith(f"[{carrier_name}]"):
            is_new_section = True
        
        # LGU+의 경우 문서 표기에 따른 예외 처리
        elif carrier_name == "LGU+" and para.startswith("[LG U+"):
            is_new_section = True

        # 새로운 섹션이 감지되면 이전까지의 내용을 하나의 리스트로 묶어 저장
        if is_new_section and current_block:
            carrier_plans[carrier_name].append("\n".join(current_block))
            current_block = []
        
        current_block.append(para)

    # 마지막 남은 블록 저장
    if current_block:
        carrier_plans[carrier_name].append("\n".join(current_block))
    
    print(f"   -> {carrier_name}: {len(carrier_plans[carrier_name])}개의 독립 섹션 추출 완료")

📄 통신사별 텍스트 분리 시작...
   -> SKT: 152개의 독립 섹션 추출 완료
   -> KT: 113개의 독립 섹션 추출 완료
   -> LGU+: 88개의 독립 섹션 추출 완료


In [40]:
# 3. 모델 로드 및 임베딩
print("\n 모델 로드 및 임베딩 시작...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

for carrier in ["SKT", "LGU+", "KT"]:
    plans = carrier_plans[carrier]
    if not plans: continue
        
    print(f"[{carrier}] 임베딩 계산 및 CSV 생성 중...")
    embeddings = model.encode(plans, batch_size=16, normalize_embeddings=True, show_progress_bar=True)
    
    # 4. CSV 데이터 생성 및 저장
    csv_data = []
    pk_prefix = "LGU" if carrier == "LGU+" else carrier
    
    for i, (full_text, embed) in enumerate(zip(plans, embeddings)):
        # 데이터 성격 판별 (INFO 인지 CONT 인지)
        is_info = True if carrier in full_text and ("통신사 :" in full_text or "가격 :" in full_text) else False
        suffix = "INFO" if is_info else "CONT"
        
        csv_data.append({
            "dense": json.dumps(embed.tolist()),
            "source": carrier,
            "file_hash": f"{carrier}_251209_final",
            "page": i + 1,
            "row": 1,
            "text": full_text,  # 요금제란? 섹션 혹은 [KT] 상세 섹션 내용 전체가 누락 없이 들어감
            "pk": f"{pk_prefix}{i + 1}_{suffix}"
        })
    
    # 통신사별 개별 파일 저장
    df = pd.DataFrame(csv_data)
    safe_name = carrier.replace("+", "plus")
    df.to_csv(f"{safe_name}_plans_final.csv", index=False, encoding='utf-8-sig')
    print(f"   -> {safe_name}_plans_final.csv 저장 완료 ({len(df)}행)")

print("\n✨ 모든 작업이 완료되었습니다! 텍스트 누락 여부를 확인하세요.")


 모델 로드 및 임베딩 시작...


C:\Users\미소정보기술\.cache\huggingface\modules\transformers_modules\nomic_hyphen_ai\nomic_hyphen_bert_hyphen_2048\7710840340a098cfb869c4f65e87cf2b1b70caca\modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +


[SKT] 임베딩 계산 및 CSV 생성 중...


Batches: 100%|██████████| 10/10 [01:34<00:00,  9.45s/it]


   -> SKT_plans_final.csv 저장 완료 (152행)
[LGU+] 임베딩 계산 및 CSV 생성 중...


Batches: 100%|██████████| 6/6 [00:41<00:00,  6.93s/it]


   -> LGUplus_plans_final.csv 저장 완료 (88행)
[KT] 임베딩 계산 및 CSV 생성 중...


Batches: 100%|██████████| 8/8 [00:55<00:00,  6.88s/it]

   -> KT_plans_final.csv 저장 완료 (113행)

✨ 모든 작업이 완료되었습니다! 텍스트 누락 여부를 확인하세요.


In [22]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [4, 4]

C:\Users\미소정보기술\.cache\huggingface\modules\transformers_modules\nomic_hyphen_ai\nomic_hyphen_bert_hyphen_2048\7710840340a098cfb869c4f65e87cf2b1b70caca\modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +


torch.Size([4, 4])


## 텍스트 파일 변환

In [41]:
import json
import os
from docx import Document
from sentence_transformers import SentenceTransformer


# 2. Markdown 변환 함수 (계층 구조 적용)
def convert_to_markdown(para_list, carrier_name):
    if not para_list: return ""
    
    # [상세 정보] 섹션 (INFO)
    if para_list[0].startswith(f"[{carrier_name}]") or "[LG U+" in para_list[0]:
        md_lines = [f"# {para_list[0]}"]
        in_features = False
        for line in para_list[1:]:
            if "요금제 특징" in line:
                md_lines.append(f"## {line}")
                in_features = True
            elif ":" in line and not in_features:
                md_lines.append(f"## {line}")
            elif in_features:
                clean = line.strip().lstrip('l ').lstrip('-').strip()
                if clean: md_lines.append(f"### {clean}")
            else:
                md_lines.append(f"### {line}")
        return "\n".join(md_lines)
    
    # [설명 섹션] (CONT)
    else:
        md_lines = []
        for line in para_list:
            if any(k in line for k in ["요금제란?", "특징", "방법", "유의사항"]):
                md_lines.append(f"## {line}")
            else:
                md_lines.append(f"### {line}")
        return "\n".join(md_lines)

# 3. 모델 로드 (임베딩 추출용)
print("🧠 모델 로드 중...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

# 4. 데이터 추출 및 .txt 저장
for name, path in PATHS:
    if not os.path.exists(path):
        print(f"⚠️ {name} 파일을 찾을 수 없습니다.")
        continue
        
    doc = Document(path)
    paras = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    
    current_block = []
    txt_output_lines = [] # .txt 파일에 저장할 행들을 담을 리스트
    pk_prefix = "LGU" if name == "LGU+" else name
    idx = 1

    print(f"[{name}] 데이터 처리 및 임베딩 중...")
    
    for para in paras:
        is_new = "요금제란?" in para or para.startswith(f"[{name}]") or (name == "LGU+" and para.startswith("[LG U+"))
        
        if is_new and current_block:
            md_text = convert_to_markdown(current_block, name)
            embed = model.encode([md_text], normalize_embeddings=True)[0]
            suffix = "INFO" if md_text.startswith("# ") else "CONT"
            
            # Attu 업로드용 JSON 구조 (한 줄의 문자열로 변환)
            entry = {
                "pk": f"{pk_prefix}{idx}_{suffix}",
                "source": name,
                "text": md_text,
                "dense": embed.tolist(),
                "page": idx,
                "row": 1
            }
            txt_output_lines.append(json.dumps(entry, ensure_ascii=False))
            
            current_block = []
            idx += 1
            
        current_block.append(para)
    
    # 마지막 블록 처리
    if current_block:
        md_text = convert_to_markdown(current_block, name)
        embed = model.encode([md_text], normalize_embeddings=True)[0]
        suffix = "INFO" if md_text.startswith("# ") else "CONT"
        entry = {"pk": f"{pk_prefix}{idx}_{suffix}", "source": name, "text": md_text, "dense": embed.tolist(), "page": idx, "row": 1}
        txt_output_lines.append(json.dumps(entry, ensure_ascii=False))

    # 5. 최종 .txt 파일 저장 (JSONL 형식: 한 줄에 하나의 JSON 객체)
    safe_name = name.replace("+", "plus")
    output_filename = f"{safe_name}_plans_final.txt"
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write("\n".join(txt_output_lines))
        
    print(f"✅ {output_filename} 저장 완료! (총 {len(txt_output_lines)}개 섹션)")

print("\n✨ 모든 통신사 .txt(JSONL) 파일 생성이 완료되었습니다.")

🧠 모델 로드 중...
[SKT] 데이터 처리 및 임베딩 중...
✅ SKT_plans_final.txt 저장 완료! (총 152개 섹션)
[KT] 데이터 처리 및 임베딩 중...
✅ KT_plans_final.txt 저장 완료! (총 113개 섹션)
[LGU+] 데이터 처리 및 임베딩 중...
✅ LGUplus_plans_final.txt 저장 완료! (총 88개 섹션)

✨ 모든 통신사 .txt(JSONL) 파일 생성이 완료되었습니다.


## json 파일 변환

In [46]:
import json
import os
from docx import Document
from sentence_transformers import SentenceTransformer


# 2. Markdown 변환 함수
def convert_to_markdown(para_list, carrier_name):
    if not para_list: return ""
    # [상세 정보] 섹션 (INFO)
    if para_list[0].startswith(f"[{carrier_name}]") or "[LG U+" in para_list[0]:
        md_lines = [f"# {para_list[0]}"]
        in_features = False
        for line in para_list[1:]:
            if "요금제 특징" in line:
                md_lines.append(f"## {line}")
                in_features = True
            elif ":" in line and not in_features:
                md_lines.append(f"## {line}")
            elif in_features:
                clean = line.strip().lstrip('l ').lstrip('-').strip()
                if clean: md_lines.append(f"### {clean}")
            else:
                md_lines.append(f"### {line}")
        return "\n".join(md_lines)
    # [설명 섹션] (CONT)
    else:
        md_lines = []
        for line in para_list:
            if any(k in line for k in ["요금제란?", "특징", "방법", "유의사항"]):
                md_lines.append(f"## {line}")
            else:
                md_lines.append(f"### {line}")
        return "\n".join(md_lines)

# 3. 모델 로드
print("🧠 모델 로드 중...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

# 4. 데이터 추출 및 .json 저장
for name, path in PATHS:
    if not os.path.exists(path):
        print(f"⚠️ {name} 파일을 찾을 수 없습니다.")
        continue
        
    doc = Document(path)
    paras = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    
    current_block = []
    json_output_list = [] # JSON 객체들을 담을 리스트
    pk_prefix = "LGU" if name == "LGU+" else name
    idx = 1

    print(f"[{name}] 데이터 처리 및 임베딩 중...")
    
    for para in paras:
        is_new = "요금제란?" in para or para.startswith(f"[{name}]") or (name == "LGU+" and para.startswith("[LG U+"))
        if is_new and current_block:
            md_text = convert_to_markdown(current_block, name)
            embed = model.encode([md_text], normalize_embeddings=True)[0]
            suffix = "INFO" if md_text.startswith("# ") else "CONT"
            
            entry = {
                "pk": f"{pk_prefix}{idx}_{suffix}",
                "source": name,
                "text": md_text,
                "dense": embed.tolist(),
                "page": idx,
                "row": 1,
                "file_hash": f"{name}_full_page_260102"
            }
            json_output_list.append(entry)
            current_block = []
            idx += 1
        current_block.append(para)
    
    # 마지막 블록 처리
    if current_block:
        md_text = convert_to_markdown(current_block, name)
        embed = model.encode([md_text], normalize_embeddings=True)[0]
        suffix = "INFO" if md_text.startswith("# ") else "CONT"
        json_output_list.append({
            "pk": f"{pk_prefix}{idx}_{suffix}", "source": name, "text": md_text, 
            "dense": embed.tolist(), "page": idx, "row": 1, "file_hash": f"{name}_full_page_260102"
        })

    # 5. 최종 .json 파일 저장
    safe_name = name.replace("+", "plus")
    output_filename = f"{safe_name}_plans_final.json"
    with open(output_filename, "w", encoding="utf-8") as f:
        # indent=4를 통해 가독성이 좋은 형태로 저장합니다.
        json.dump(json_output_list, f, ensure_ascii=False, indent=4)
        
    print(f"✅ {output_filename} 저장 완료! (총 {len(json_output_list)}개 객체)")

print("\n✨ 모든 통신사 .json 파일 생성이 완료되었습니다.")

🧠 모델 로드 중...
[SKT] 데이터 처리 및 임베딩 중...
✅ SKT_plans_final.json 저장 완료! (총 152개 객체)
[KT] 데이터 처리 및 임베딩 중...
✅ KT_plans_final.json 저장 완료! (총 113개 객체)
[LGU+] 데이터 처리 및 임베딩 중...
✅ LGUplus_plans_final.json 저장 완료! (총 88개 객체)

✨ 모든 통신사 .json 파일 생성이 완료되었습니다.


# MD 방식으로 청킹하기

In [66]:
import json
import os
from docx import Document
from sentence_transformers import SentenceTransformer

# 1. 설정 및 경로
SOURCE_PATH = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\요금제RAG\KT요금제정보_251209.docx"
OUTPUT_PATH = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\KT_plans_final.json"

# 8,000 토큰을 수용하기 위한 글자 수 설정 (한글 기준 약 12,000 ~ 15,000자)
MAX_CHARS_PER_CHUNK = 500

print("🧠 모델 로드 및 대용량 청킹 시작...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

def format_to_md(unit_paras):
    """사용자 지정 규칙에 따른 MD 계층화"""
    if not unit_paras: return ""
    
    md_lines = [f"# {unit_paras[0]}"] # 첫 줄 H1
    section_headers = ["OTT 혜택", "구독 혜택", "부가 혜택", "요금제 특징"]
    
    for line in unit_paras[1:]:
        # 규칙 1: 지정된 핵심 섹션 헤더는 ##
        if any(header in line for header in section_headers):
            md_lines.append(f"## {line}")
        # 규칙 2: 콜론(:)이 포함된 속성 항목은 ##
        elif ":" in line:
            md_lines.append(f"## {line}")
        # 규칙 3: 그 외의 나열된 특징 및 본문은 ###
        else:
            md_lines.append(f"### {line}")
    return "\n".join(md_lines)

def generate_rag_json():
    if not os.path.exists(SOURCE_PATH):
        print(f"❌ 파일을 찾을 수 없습니다: {SOURCE_PATH}")
        return

    doc = Document(SOURCE_PATH)
    paras = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    
    # [Step 1] 요금제 유닛 분리
    units = []
    current_unit = []
    for para in paras:
        if "요금제란?" in para or para.startswith("[KT]"):
            if current_unit:
                units.append(format_to_md(current_unit))
                current_unit = []
        current_unit.append(para)
    if current_unit:
        units.append(format_to_md(current_unit))

    # [Step 2] 8,000 토큰 규모로 병합
    final_chunks = []
    temp_content = []
    temp_length = 0

    for md_text in units:
        # 병합 시 글자 수 기준 체크
        if temp_length + len(md_text) > MAX_CHARS_PER_CHUNK and temp_content:
            final_chunks.append("\n\n---\n\n".join(temp_content))
            temp_content = [md_text]
            temp_length = len(md_text)
        else:
            temp_content.append(md_text)
            temp_length += (len(md_text) + 10)

    if temp_content:
        final_chunks.append("\n\n---\n\n".join(temp_content))

    # [Step 3] JSON 구성 및 임베딩
    json_data = []
    print(f"📊 총 {len(final_chunks)}개의 대형 청크 생성됨. 임베딩 중...")
    
    for i, chunk_text in enumerate(final_chunks):
        dense_vec = model.encode([chunk_text], normalize_embeddings=True)[0]
        
        json_data.append({
            "pk": f"KT_CHUNK_{i+1}",
            "source": "KT",
            "text": chunk_text,
            "dense": dense_vec.tolist(),
            "file_hash": "KT_260107_8K_TOKEN_ALIGNED",
            "page": i + 1,
            "row": 1
        })

    # [Step 4] 파일 저장
    os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
    with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)
    
    print(f"✅ 성공: {OUTPUT_PATH} 저장 완료!")

if __name__ == "__main__":
    generate_rag_json()

🧠 모델 로드 및 대용량 청킹 시작...


C:\Users\미소정보기술\.cache\huggingface\modules\transformers_modules\nomic_hyphen_ai\nomic_hyphen_bert_hyphen_2048\7710840340a098cfb869c4f65e87cf2b1b70caca\modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +


📊 총 113개의 대형 청크 생성됨. 임베딩 중...
✅ 성공: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\KT_plans_final.json 저장 완료!


In [63]:
import json
import os
from docx import Document
from sentence_transformers import SentenceTransformer

# 1. 경로 및 설정
SOURCE_PATH = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\요금제RAG\KT요금제정보_251209.docx"
OUTPUT_JSON_PATH = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\KT_plans_optimized.json"
MAX_CHARS_PER_CHUNK = 2000 

print("🧠 모델 로드 및 최적화 청킹 시작...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

def format_to_md(unit_paras):
    """지정된 규칙에 따라 Markdown 계층 구조 생성"""
    if not unit_paras: return ""
    
    md_lines = []
    # 1. 첫 줄은 무조건 # (H1)
    md_lines.append(f"# {unit_paras[0]}")
    
    # 하위 내용을 ###로 강제해야 하는 '섹션 헤더' 키워드 정의
    section_headers = ["OTT 혜택", "구독 혜택", "부가 혜택", "요금제 특징"]
    
    current_section_is_list = False # 현재 섹션이 리스트형 섹션인지 추적

    for line in unit_paras[1:]:
        # 규칙 A: 섹션 헤더 키워드인 경우 ## (H2)
        if any(header in line for header in section_headers):
            md_lines.append(f"## {line}")
            current_section_is_list = True # 이후 나오는 내용들은 ###로 처리
        
        # 규칙 B: 콜론(:)이 포함된 일반 속성인 경우 ## (H2)
        elif ":" in line:
            md_lines.append(f"## {line}")
            current_section_is_list = False # 속성값이므로 리스트 모드 해제
            
        # 규칙 C: 그 외의 세부 내용이나 특징 리스트인 경우 ### (H3)
        else:
            md_lines.append(f"### {line}")

    return "\n".join(md_lines)

def generate_optimized_rag_json():
    if not os.path.exists(SOURCE_PATH):
        print(f"❌ 파일을 찾을 수 없습니다: {SOURCE_PATH}")
        return

    doc = Document(SOURCE_PATH)
    paras = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    
    # [Step 1] 요금제 유닛 분리
    units = []
    current_unit = []
    for para in paras:
        if "요금제란?" in para or para.startswith("[KT]"):
            if current_unit:
                units.append(format_to_md(current_unit))
                current_unit = []
        current_unit.append(para)
    if current_unit:
        units.append(format_to_md(current_unit))

    # [Step 2] 글자 수 기반 병합
    final_chunks_text = []
    temp_content = []
    temp_length = 0

    for md_text in units:
        if temp_length + len(md_text) > MAX_CHARS_PER_CHUNK and temp_content:
            final_chunks_text.append("\n\n---\n\n".join(temp_content))
            temp_content = [md_text]
            temp_length = len(md_text)
        else:
            temp_content.append(md_text)
            temp_length += len(md_text)

    if temp_content:
        final_chunks_text.append("\n\n---\n\n".join(temp_content))

    # [Step 3] JSON 구성
    json_data = []
    for i, chunk_text in enumerate(final_chunks_text):
        dense_vec = model.encode([chunk_text], normalize_embeddings=True)[0]
        json_data.append({
            "pk": f"KT_OPT_{i+1}",
            "source": "KT",
            "text": chunk_text,
            "dense": dense_vec.tolist(),
            "file_hash": "KT_260107_MD_FIXED",
            "page": i + 1,
            "row": 1
        })

    os.makedirs(os.path.dirname(OUTPUT_JSON_PATH), exist_ok=True)
    with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)
    
    print(f"✅ 완료: {OUTPUT_JSON_PATH} 저장됨.")

if __name__ == "__main__":
    generate_optimized_rag_json()

🧠 모델 로드 및 최적화 청킹 시작...


C:\Users\미소정보기술\.cache\huggingface\modules\transformers_modules\nomic_hyphen_ai\nomic_hyphen_bert_hyphen_2048\7710840340a098cfb869c4f65e87cf2b1b70caca\modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +


✅ 완료: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\KT_plans_optimized.json 저장됨.


# 7개로 각각 다른 사이즈로 청킹

In [53]:
import json
import os
import math
from docx import Document
from sentence_transformers import SentenceTransformer

# 1. 경로 설정
SOURCE_PATH = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\요금제RAG\SKT요금제정보_251209.docx"
OUTPUT_JSON_PATH = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\SKT_plans_final_7chunks.json"

print("🧠 모델 로드 및 7개 청크 분할 시작...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

def generate_7_chunks_json():
    if not os.path.exists(SOURCE_PATH):
        print(f"❌ 파일을 찾을 수 없습니다: {SOURCE_PATH}")
        return

    doc = Document(SOURCE_PATH)
    paras = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    
    # [Step 1] 모든 요금제 섹션을 개별 유닛으로 추출 (총 113개)
    units = []
    current_unit = []
    for para in paras:
        if "요금제란?" in para or para.startswith("[KT]"):
            if current_unit:
                units.append(current_unit)
                current_unit = []
        current_unit.append(para)
    if current_unit:
        units.append(current_unit)

    print(f"📊 총 {len(units)}개의 요금제 유닛 확인됨. 7개 청크로 분할합니다.")

    # [Step 2] 유닛들을 MD 서식으로 변환
    md_units = []
    for unit in units:
        md_lines = [f"# {unit[0]}"] # 요금제 시작 (#)
        for line in unit[1:]:
            if ":" in line:
                md_lines.append(f"## {line}") # 속성 (##)
            else:
                md_lines.append(f"### {line}") # 특징/설명 (###)
        md_units.append("\n".join(md_lines))

    # [Step 3] 7개의 균등한 청크로 그룹화
    num_chunks = 7
    total_units = len(md_units)
    # 각 청크에 들어갈 유닛 수 계산 (나머지 처리 포함)
    units_per_chunk = total_units // num_chunks
    remainder = total_units % num_chunks

    final_chunks_text = []
    start_idx = 0
    for i in range(num_chunks):
        # 나머지 개수만큼 앞에서부터 1개씩 더 배정 (17개씩 1번, 나머지는 16개씩)
        end_idx = start_idx + units_per_chunk + (1 if i < remainder else 0)
        chunk_group = md_units[start_idx:end_idx]
        # 구분자(---)로 유닛 결합
        final_chunks_text.append("\n\n---\n\n".join(chunk_group))
        start_idx = end_idx

    # [Step 4] JSON 구성 및 임베딩
    json_data = []
    for i, chunk_text in enumerate(final_chunks_text):
        print(f"📡 청크 {i+1} 임베딩 중... (약 {len(chunk_text)}자)")
        dense_vec = model.encode([chunk_text], normalize_embeddings=True)[0]
        
        json_data.append({
            "pk": f"SKT_CHUNK_{i+1}",
            "source": "SKT",
            "text": chunk_text,
            "dense": dense_vec.tolist(),
            "file_hash": "SKT_260107_7CHUNKS_MD",
            "page": i + 1,
            "row": 1
        })

    # [Step 5] 파일 저장
    os.makedirs(os.path.dirname(OUTPUT_JSON_PATH), exist_ok=True)
    with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)
    
    print(f"✅ 완료: 총 {len(json_data)}개의 통합 청크가 {OUTPUT_JSON_PATH}에 저장되었습니다.")

generate_7_chunks_json()

🧠 모델 로드 및 7개 청크 분할 시작...


C:\Users\미소정보기술\.cache\huggingface\modules\transformers_modules\nomic_hyphen_ai\nomic_hyphen_bert_hyphen_2048\7710840340a098cfb869c4f65e87cf2b1b70caca\modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +


📊 총 23개의 요금제 유닛 확인됨. 7개 청크로 분할합니다.
📡 청크 1 임베딩 중... (약 31756자)
📡 청크 2 임베딩 중... (약 15660자)
📡 청크 3 임베딩 중... (약 5978자)
📡 청크 4 임베딩 중... (약 7598자)
📡 청크 5 임베딩 중... (약 6097자)
📡 청크 6 임베딩 중... (약 5890자)
📡 청크 7 임베딩 중... (약 4193자)
✅ 완료: 총 7개의 통합 청크가 C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\SKT_plans_final_7chunks.json에 저장되었습니다.


# 청크 균등사이즈로 분할
- SKT,KG, LGU+ 케이스별로 글자 바꿀것

In [55]:
import json
import os
from docx import Document
from sentence_transformers import SentenceTransformer

# 1. 경로 설정
SOURCE_PATH = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\요금제RAG\KT요금제정보_251209.docx"
OUTPUT_JSON_PATH = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\KT_plans_final_balanced.json"

print("🧠 모델 로드 및 균등 비율 청킹 시작...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

def generate_balanced_7_chunks():
    if not os.path.exists(SOURCE_PATH):
        print(f"❌ 파일을 찾을 수 없습니다: {SOURCE_PATH}")
        return

    doc = Document(SOURCE_PATH)
    paras = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    
    # [Step 1] 요금제 유닛 추출 및 MD 변환
    md_units = []
    current_unit = []
    for para in paras:
        if "요금제란?" in para or para.startswith("[KT]"):
            if current_unit:
                md_units.append(format_to_md(current_unit))
                current_unit = []
        current_unit.append(para)
    if current_unit:
        md_units.append(format_to_md(current_unit))

    # [Step 2] 전체 글자 수 계산 및 목표 청크 크기 설정
    total_length = sum(len(u) for u in md_units)
    target_chunk_size = total_length // 7
    print(f"📊 전체 글자 수: {total_length}자 / 목표 청크 크기: 약 {target_chunk_size}자")

    # [Step 3] 글자 수 기반 균등 배분
    final_chunks_text = []
    current_chunk_content = []
    current_chunk_length = 0

    for unit in md_units:
        # 현재 청크에 추가했을 때 목표 크기에 가장 가까워지도록 배분
        # (이미 데이터가 있고, 추가 시 목표를 크게 상회하면 다음 청크로 넘김)
        if current_chunk_length > 0 and current_chunk_length + len(unit) > target_chunk_size + 1000:
            if len(final_chunks_text) < 6: # 7개까지만 생성하도록 제어
                final_chunks_text.append("\n\n---\n\n".join(current_chunk_content))
                current_chunk_content = [unit]
                current_chunk_length = len(unit)
                continue
        
        current_chunk_content.append(unit)
        current_chunk_length += len(unit)

    # 마지막 남은 내용 처리
    if current_chunk_content:
        final_chunks_text.append("\n\n---\n\n".join(current_chunk_content))

    # [Step 4] JSON 구성 및 임베딩
    json_data = []
    for i, chunk_text in enumerate(final_chunks_text):
        print(f"📡 청크 {i+1} 임베딩 중... (길이: {len(chunk_text)}자)")
        dense_vec = model.encode([chunk_text], normalize_embeddings=True)[0]
        
        json_data.append({
            "pk": f"KT_CHUNK_{i+1}",
            "source": "KT",
            "text": chunk_text,
            "dense": dense_vec.tolist(),
            "file_hash": "KT_260107_BALANCED_7CHUNKS",
            "page": i + 1,
            "row": 1
        })

    # [Step 5] 저장
    os.makedirs(os.path.dirname(OUTPUT_JSON_PATH), exist_ok=True)
    with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)
    print(f"✅ 완료: {len(json_data)}개의 균등 청크가 저장되었습니다.")

def format_to_md(unit):
    """유닛 리스트를 MD 형식 텍스트로 변환"""
    md_lines = [f"# {unit[0]}"]
    for line in unit[1:]:
        md_lines.append(f"## {line}" if ":" in line else f"### {line}")
    return "\n".join(md_lines)

generate_balanced_7_chunks()

🧠 모델 로드 및 균등 비율 청킹 시작...
📊 전체 글자 수: 59186자 / 목표 청크 크기: 약 8455자
📡 청크 1 임베딩 중... (길이: 9485자)
📡 청크 2 임베딩 중... (길이: 9184자)
📡 청크 3 임베딩 중... (길이: 9246자)
📡 청크 4 임베딩 중... (길이: 9369자)
📡 청크 5 임베딩 중... (길이: 9547자)
📡 청크 6 임베딩 중... (길이: 9461자)
📡 청크 7 임베딩 중... (길이: 3636자)
✅ 완료: 7개의 균등 청크가 저장되었습니다.


# 통신사_test_v3 파일로 MD 청킹 

In [1]:
import json
import os
from docx import Document
from sentence_transformers import SentenceTransformer

# 1. 경로 및 설정
SOURCE_PATH = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\시나리오2번용RAG\SKT_test_v3.docx"
OUTPUT_JSON_PATH = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\json폴더\SKT_simple_optimized.json"

# 최적 청크 찾기
MAX_CHARS_PER_CHUNK = 1000

print("🧠 심플 MD 변환 및 임베딩 시작...")
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)

def format_simple_to_md(unit_paras):
    """[요금제 시작] 기반 데이터를 ## 형식의 MD로 변환"""
    if not unit_paras: return ""
    
    md_lines = []
    for line in unit_paras:
        # '[요금제 시작]' 문구는 제외하고 실제 정보만 담습니다.
        if "[요금제 시작]" in line:
            continue
        
        # 모든 항목 앞에 ##를 붙여서 일관된 계층 구조 형성
        if ":" in line:
            md_lines.append(f"## {line}")
        else:
            # 혹시 콜론이 없는 줄이 있다면 상세 내용으로 간주하여 ### 처리
            md_lines.append(f"### {line}")
            
    return "\n".join(md_lines)

def generate_simple_rag_json():
    if not os.path.exists(SOURCE_PATH):
        print(f"❌ 파일을 찾을 수 없습니다: {SOURCE_PATH}")
        return

    doc = Document(SOURCE_PATH)
    paras = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    
    # [Step 1] '[요금제 시작]' 키워드로 요금제별 유닛 분리
    units = []
    current_unit = []
    for para in paras:
        if "[요금제 시작]" in para:
            if current_unit:
                units.append(format_simple_to_md(current_unit))
                current_unit = []
        current_unit.append(para)
    if current_unit:
        units.append(format_simple_to_md(current_unit))

    # [Step 2] 설정한 글자 수(2000자)에 맞춰 병합
    final_chunks_text = []
    temp_content = []
    temp_length = 0

    for md_text in units:
        if temp_length + len(md_text) > MAX_CHARS_PER_CHUNK and temp_content:
            final_chunks_text.append("\n\n---\n\n".join(temp_content))
            temp_content = [md_text]
            temp_length = len(md_text)
        else:
            temp_content.append(md_text)
            temp_length += (len(md_text) + 10) # 구분자 간격 고려

    if temp_content:
        final_chunks_text.append("\n\n---\n\n".join(temp_content))

    # [Step 3] JSON 구성 및 임베딩
    json_data = []
    print(f"📊 총 {len(final_chunks_text)}개의 심플 청크 생성 완료. 임베딩 중...")
    
    for i, chunk_text in enumerate(final_chunks_text):
        dense_vec = model.encode([chunk_text], normalize_embeddings=True)[0]
        
        json_data.append({
            "pk": f"SKT_SIM_{i+1}",
            "source": "SKT",
            "text": chunk_text,
            "dense": dense_vec.tolist(),
            "file_hash": "SKT_V3_SIMPLE_MD",
            "page": i + 1,
            "row": 1
        })

    # [Step 4] 저장
    os.makedirs(os.path.dirname(OUTPUT_JSON_PATH), exist_ok=True)
    with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)
    
    print(f"✅ 완료: {OUTPUT_JSON_PATH}에 저장되었습니다.")

if __name__ == "__main__":
    generate_simple_rag_json()

🧠 심플 MD 변환 및 임베딩 시작...


C:\Users\미소정보기술\.cache\huggingface\modules\transformers_modules\nomic_hyphen_ai\nomic_hyphen_bert_hyphen_2048\7710840340a098cfb869c4f65e87cf2b1b70caca\modeling_hf_nomic_bert.py:1634: UserWarning: Install Nomic's megablocks fork for better speed: `pip install git+https://github.com/nomic-ai/megablocks.git`
  warnings.warn("Install Nomic's megablocks fork for better speed: " +


📊 총 29개의 심플 청크 생성 완료. 임베딩 중...
✅ 완료: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\json폴더\SKT_simple_optimized.json에 저장되었습니다.
